In [45]:
import numpy as np
import pandas as pd
from itertools import product
from sklearn.preprocessing import normalize

#### Participant simulation

First we'll create *n* participants in our experiment.

In [46]:
n = 1000

For each of these participants, we'll randomly assign them some (Compliance, Response) behavior combination.

In [47]:
compliance_partitions = ['always_taker', 
                         'complier', 
                         'defier', 
                         'never_taker']
response_partitions = ['always_better', 
                       'helped', 
                       'hurt', 
                       'never_better']

# Take the cross product of these sets of types

partition_types = product(compliance_partitions, response_partitions)
partition_types = np.array(list(partition_types))


In [48]:
# We'll simulate probabilities that our participants 
# will belong to one of the 16 possible behavior combinations

random_floats = np.random.random([1,16]) 
partition_probabilities = normalize(random_floats, 'l1')
partition_probabilities = partition_probabilities.flatten()

# to be a true set of probabilities, the vector sum 
# needs to be 1

# sometimes this can fail because of precision errors, 
# so let's assert it

assert partition_probabilities.sum() == 1 

In [49]:
# drawing participant compliance and response behaviors according to the
# specified distribution

participant_partition = np.random.choice(
                            range(len(partition_types)), 
                            n, 
                            p=partition_probabilities
                        )

compliance_response_pairs = zip(*partition_types[participant_partition])
compliance_type, response_type = list(compliance_response_pairs)

# assigning participants to Control and Treatment groups 
# with 50% probability

assignments = ['control', 'treatment']
participant_assignment = np.random.choice(assignments, n)

# compiling all information into a dataframe
# that simulates the participants

df = pd.DataFrame({'assignment': participant_assignment,
                   'compliance_type': compliance_type,
                   'response_type': response_type})

### Simulate whether participants took treatment

Depending on assignment and compliance type, we can simulate whether or not each participant took the treatment.

In [50]:
# if the participant is an always_taker, 
# they'll always take the treatment.

df['took_treatment'] = (df.compliance_type == 'always_taker')

# if they're a complier, they'll take the treatment 
# as long as they're in the treatment condition.

df['took_treatment'] = df['took_treatment'] \
                        | ( 
                            (df.compliance_type == 'complier') 
                            & 
                            (df.assignment == 'treatment') 
                          )

# if they're a defier, they'll only take the treatment 
# if they were in the control condition.

df['took_treatment'] = df['took_treatment'] \
                        | ( 
                            (df.compliance_type == 'defier') 
                            & 
                            (df.assignment == 'control') 
                          )

### Simulate Outcomes

Now we can simulate outcomes from the experiment.

Depending on whether they took the treatment and their `response_type`, did they end up in a Good or Bad state after the experiment's conclusion?

In [51]:
# if the participant is of the always_better type, 
# they'll definitely have a good outcome.

df['good_outcome'] = (df.response_type == 'always_better') 

# if the participant is of the 'helped' type, 
# they'll have a good outcome as long as they
# took treatment.

df['good_outcome'] = df['good_outcome'] \
                     | ( 
                        (df.response_type == 'helped') 
                        & 
                        (df.took_treatment) 
                       )

# Otherwise, the outcome is going to be bad
# and the column will have a False value.

We can now observe the probabilities of each (Treatment, Outcome) combination that would emerge, conditional on the assignment.

In [52]:
df['n'] = 1
results = df.groupby(['assignment', 
                      'took_treatment', 
                      'good_outcome']).count().n
results = results.to_frame()
results['assignment_n'] = results.groupby('assignment').transform('sum')

In [53]:
p_states = results.n / results.assignment_n
p_states = p_states.rename("P( X, Y | Z )")

# display
p_states.to_frame()

P( X, Y | Z )
assignment took_treatment good_outcome               
control    False          False              0.422764
                          True               0.117886
           True           False              0.308943
                          True               0.150407
treatment  False          False              0.397638
                          True               0.124016
           True           False              0.240157
                          True               0.238189

### Intent-to-treat average treatment effect

All we need to do is sum up all 'good' from the Treatment column, and subtract by all 'good' from the Control column.


In [67]:
itt_ate = df[df.assignment == 'treatment'].good_outcome.mean() \
          - df[df.assignment == 'control'].good_outcome.mean() 

# display
print("ATE: %6.4f" % itt_ate)

ATE: 0.0939


### Linear optimization

First we'll get all the probabilities we need: p(x,y|z) for each x,y,z combination:

In [79]:
states = list(product(['treatment','control'],[0,1], [0,1]))

get_conditional_probability = lambda assignment, took, outcome: \
                ( 
                    (df[df.assignment == assignment].took_treatment == took)
                    & 
                    (df[df.assignment == assignment].good_outcome == outcome)  
                ).mean()

# this terrible list comprehension
# collects p(x,y|z) for all 'states'
p_states = {
            f"{assignment}/"
            + f"{'treated' if took == 1 else 'untreated'}/"
            + f"{'good' if outcome ==1 else 'bad'}" : 
                get_conditional_probability(assignment, took, outcome)
                for assignment, took, outcome in states
            }

In [80]:
p_states

{'treatment/untreated/bad': 0.39763779527559057,
 'treatment/untreated/good': 0.12401574803149606,
 'treatment/treated/bad': 0.24015748031496062,
 'treatment/treated/good': 0.23818897637795275,
 'control/untreated/bad': 0.42276422764227645,
 'control/untreated/good': 0.11788617886178862,
 'control/treated/bad': 0.3089430894308943,
 'control/treated/good': 0.15040650406504066}

Now we can frame our linear programming problem.

We're minimizing/maximizing our expression of an ATE given the constraints at hand.

In [111]:
import pulp

# first we set up 'problems', objects that
# can take an objective like 'maximize' as well as constraints

max_problem = pulp.LpProblem("max ATE", pulp.LpMaximize)

We can now specify variables, symbols that the linear program will know to vary in order to maximize or minimize our objective function. These variables are distribution of **U**: the probabilities of belonging to one of the archetypal (Compliance, Response) partitions.

In [112]:
# Linear optimization ranges over possible variables
# to find those hidden variables that maximize
# or minimize our objective.

# Our variables 'q' are the the probability of 
# being in one of the partitions

partition_names = ['/'.join([compliance, response]) 
                       for compliance, response in partition_types]

q = {partition: pulp.LpVariable(partition, lowBound=0) for partition in partition_names}

We can add constraints on these variables as well, by simply adding them to the problem we've framed!

In [113]:
# since our hidden vars are probabilities 
# the sum of them should all be under 1

max_problem += sum([v for k,v in q.items()]) == 1

In [114]:
# Let's set the relationship between the distribution of U.
# and the probabilities we observe. The variable name
# scheme is p_Z_X_Y


p_treatment_untreated_bad = q['never_taker/never_better'] \
                            + q['defier/never_better'] \
                            + q['never_taker/helped'] \
                            + q['defier/helped'] 

p_treatment_untreated_good = q['never_taker/always_better'] \
                             + q['defier/always_better'] \
                             + q['never_taker/hurt'] \
                             + q['defier/hurt']

p_treatment_treated_bad = q['always_taker/never_better'] \
                          + q['complier/never_better'] \
                          + q['always_taker/hurt'] \
                          + q['complier/hurt']

p_treatment_treated_good = q['always_taker/always_better'] \
                           + q['complier/always_better'] \
                           + q['always_taker/helped'] \
                           + q['complier/helped']

p_control_untreated_bad = q['never_taker/never_better'] \
                          + q['complier/never_better'] \
                          + q['never_taker/helped'] \
                          + q['complier/helped']

p_control_untreated_good = q['never_taker/always_better'] \
                           + q['complier/never_better'] \
                           + q['never_taker/hurt'] \
                           + q['complier/hurt']

p_control_treated_bad = q['always_taker/never_better'] \
                        + q['defier/never_better'] \
                        + q['always_taker/hurt'] \
                        + q['defier/hurt']

p_control_treated_good = q['always_taker/always_better'] \
                         + q['defier/always_better'] \
                         + q['always_taker/helped'] \
                         + q['defier/helped']

In [115]:
# I now apply these linear relationships between U
# and P(X,Y|Z) as constraints on the LP problem.

# Though there's a compact and elegant linear algebraic way
# of doing this, I'm being painfully explicit here because
# it's easier to understand (for me at least)

max_problem += p_treatment_untreated_bad == p_states['treatment/untreated/bad']
max_problem += p_treatment_untreated_good == p_states['treatment/untreated/good']
max_problem += p_treatment_treated_bad == p_states['treatment/treated/bad']
max_problem += p_control_untreated_bad == p_states['control/untreated/bad']
max_problem += p_control_untreated_good == p_states['control/untreated/good']
max_problem += p_control_treated_bad == p_states['control/treated/bad']


# I leave some constraints out because it *over*constrains 
# the problem and makes it impossible to solve
# It turns out that the other constraints actually imply 
# these two, so we can just leave them commented out.

#max_problem += p_treatment_treated_good == p_states['treatment/treated/good']
#max_problem += p_control_treated_good == p_states['control/treated/good']

With constraints set, all that remains to do set the objective function, our ATE.

$$ATE = P(helped) - P(hurt)$$

where $P(helped)$ is the probability of being in a partition with a 'helped' response type and $P(hurt)$ is the probability of being in a partition with a 'hurt' response type.

Then we can just hit `solve()`.

In [116]:
max_problem += q['complier/helped'] \
               + q['defier/helped'] \
               + q['always_taker/helped'] \
               + q['never_taker/helped'] \
               - q['complier/hurt'] \
               - q['defier/hurt'] \
               - q['always_taker/hurt'] \
               - q['never_taker/hurt']

In [119]:
pulp.LpStatus[max_problem.solve()]

'Optimal'

The problem's been optimized, and now we can see what U looks like in the best case scenario.

In [127]:
q_max = {partition:pulp.value(partition_p) 
         for partition, partition_p in q.items()}

# display

pd.DataFrame(q_max, index=['p(U=u)']).T

,p(U=u)
always_taker/always_better,0.000000
always_taker/helped,0.000000
always_taker/hurt,0.000000
always_taker/never_better,0.240157
complier/always_better,0.000000
complier/helped,0.238189
complier/hurt,0.000000
complier/never_better,0.000000
defier/always_better,0.006130
defier/helped,0.144277


Given these we can calculate our best-case ATE.

In [129]:
ate = lambda q: q['complier/helped'] \
               + q['defier/helped'] \
               + q['always_taker/helped'] \
               + q['never_taker/helped'] \
               - q['complier/hurt'] \
               - q['defier/hurt'] \
               - q['always_taker/hurt'] \
               - q['never_taker/hurt']

best_case_ate = ate(q_max)

# display
print("Best-Case ATE: %6.4f" % best_case_ate)

Best-Case ATE: 0.5670
